In [ ]:
from dotenv import load_dotenv

from langchain.chat_models import init_chat_model
from langchain_core.messages import AIMessage
from langchain_core.messages import HumanMessage

load_dotenv()

model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [ ]:
ai_message = model.invoke([HumanMessage(content="Hi! I'm Dan.")])

print(ai_message.content)

Hi Dan! It's great to meet you. How can I help you today?


In [7]:
ai_message = model.invoke([
    HumanMessage(content="Hi! I'm Dan."),
    AIMessage(content="Hi Dan! It's great to meet you. How can I help you today?"),
    HumanMessage(content="What's my name?"),
])

print(ai_message.content)

Your name is Dan! You told me that in your first message. 😊


In [5]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [12]:
config = { "configurable": {"thread_id": "abc123"}}

In [6]:
output = app.invoke({"messages": [HumanMessage(content="Hi! I'm Dan.")]}, config)

output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hi Dan! It's great to meet you. How can I help you today?


In [13]:
output = app.invoke({"messages": [HumanMessage(content="What's my name?")]}, config)

output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is still Dan! You told me earlier, and I remember. 😊

Is there a particular reason you're asking again?
